In [1]:
import _init_paths
from fast_rcnn.train import get_training_roidb, train_net
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from datasets.factory import get_imdb
import datasets.imdb
import caffe
import argparse
import pprint
import numpy as np
import sys
import zl_config as C
from fast_rcnn.test import im_detect
import matplotlib.pyplot as plt
from fast_rcnn.nms_wrapper import nms

CLASSES = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')

In [2]:
def combined_roidb(imdb_names):
    def get_roidb(imdb_name):
        imdb = get_imdb(imdb_name)
        print 'Loaded dataset `{:s}` for training'.format(imdb.name)
        imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
        print 'Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD)
        roidb = get_training_roidb(imdb)
        return roidb

    roidbs = [get_roidb(s) for s in imdb_names.split('+')]
    roidb = roidbs[0]
    if len(roidbs) > 1:
        for r in roidbs[1:]:
            roidb.extend(r)
        imdb = datasets.imdb.imdb(imdb_names)
    else:
        imdb = get_imdb(imdb_names)
    return imdb, roidb


cfg_from_file('experiments/cfgs/rfcn_end2end.yml')

In [3]:
imdb, roidb = combined_roidb('voc_0712_test')

Loaded dataset `voc_0712_test` for training
Set proposal method: gt
Appending horizontally-flipped training examples...


voc_0712_test gt roidb loaded from /home/zawlin/g/iccv2017/data/cache/voc_0712_test_gt_roidb.pkl
done
Preparing training data...


I0115 23:02:42.793850 28587 roidb.py:31] loading roidb from /media/zawlin/ssd/iccv2017/_cache/voc_0712_test.roidb.pkl


done


In [9]:
import cv2

ann = roidb[9]
im = cv2.imread(ann['image'])
idx = 0
for bb in ann['boxes']:   
    cv2.rectangle(im,(bb[0],bb[1]),(bb[2],bb[3]),(0,255,0),1)
cv2.imshow('im2',im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
net =None
cfg.TEST.HAS_RPN=False
prototxt = 'models/pascal_voc/ResNet-50/rfcn_end2end/test_no_rpn.prototxt'
model = 'data/rfcn_models/resnet50_rfcn_iter_800.caffemodel'
caffe.set_mode_gpu()
caffe.set_device(0)
net = caffe.Net(prototxt, model, caffe.TEST)

In [11]:
print ann['image']

#im = cv2.imread('data/demo/004545.jpg')
im = cv2.imread(ann['image'])
print ann['boxes']
scores, boxes = im_detect(net, im,boxes=ann['boxes'])

/home/zawlin/g/iccv2017/data/VOCdevkit0712/VOC0712/JPEGImages/000001.jpg
[[ 47 239 194 370]
 [  7  11 351 497]]


Exception: Input blob arguments do not match net inputs.

In [16]:
print boxes

[[  82.17454529    0.          381.35394287  333.           22.0294342
    38.55183411  377.24401855  333.        ]]


In [18]:
cls_score = net.blobs['cls_score'].data.copy()
cls_score_reindexed_caffe= net.blobs['cls_score_reindexed'].data.copy()
vatt_caffe = net.blobs['vatt'].data.copy()
cls_score_tiled_caffe= net.blobs['cls_score_tiled'].data.copy()
cls_score_tiled_transposed_caffe = net.blobs['cls_score_tiled_transposed'].data.copy()
vatt_raw_caffe = net.blobs['vatt_raw'].data.copy()
attention_caffe = net.blobs['attention'].data.copy()
attention_tiled_caffe = net.blobs['attention_tiled'].data.copy()
cls_score_tiled_caffe = net.blobs['cls_score_tiled'].data.copy()

cls_score_transposed = cls_score.transpose((1,0,2,3))
cls_score_reindexed = cls_score_transposed[15,...]

attention = softmax(cls_score_reindexed.squeeze())
rois = net.blobs['rois'].data
rois = rois/net.blobs['im_info'].data[0,2]
roi_scores = net.blobs['rois_score'].data

vatt = np.zeros((rois.shape[0],21,1,1),np.float32)
for i in xrange(vatt.shape[0]):
    vatt[i] += attention[i] * cls_score[i]
#vatt = vatt.sum(axis=0)
vatt_summed= vatt.sum(axis=0)
attention = net.blobs['attention'].data[:,0].squeeze()
ind = np.argsort(attention)[::-1]
attention = attention[ind]
rois_all = np.hstack((rois[:,1:],roi_scores))
rois_all =  rois_all[ind]
for i in xrange(5):
    ascore = attention[i]
    roi = rois_all[i]
    cv2.rectangle(im,(roi[0],roi[1]),(roi[2],roi[3]),(255,0,0),1)

cv2.imshow('im',im)
cv2.waitKey(0)
timer.toc()

[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    7.70097759e-05]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    2.33336483e-04]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    9.99422468e-07]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    2.59845547e-06]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    2.33879509e-05]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    9.89162686e-07]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    4.41333192e-04]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    5.21125614e-08]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    7.75240005e-06]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    4.70429768e-06]]
[[  2.20294342e+01   3.85518341e+01   3.77244019e+02   3.33000000e+02
    6.90469847e-07]]

In [11]:
net=None